## Одномерный метод Ньютона

_Иванычев Сергей, 376 группа_


In [51]:
import numpy as np
import sympy as sy
from sympy.utilities import lambdify
from math import pi
from scipy.optimize import minimize

In [15]:
x = sy.symbols('x')
sym_f = sy.sin(x)
sym_df = sy.diff(f, x)
sym_ddf = sy.diff(df, x)
f = lambdify(x, sym_f)
df = lambdify(x, sym_df)
ddf = lambdify(x, sym_ddf)

In [44]:
def newton_1d(x_0, f, df, ddf, eps=0.0001, iter_max=1000000, verbose=True):
    """
    Input:
    x_0         starting point
    f           target function
    df          first derivative
    ddf         second derivative
    eps         stopping treshold for gradient norm
    iter_max    maximum number of iterations
    verbose     enabling text output
    
    Output:
    function extremum value, extremum point
    """
    if verbose == True:
        vprint = print
    else:
        def vprint(*args, **kwargs):
            return None
    iters = 0
    x = x_0
    x_old = None
    while iters < iter_max and np.abs(df(x)) > eps:
        x = x - df(x)/ddf(x)
        iters += 1
    vprint("Iterations = %d" % iters)
    return f(x), x

In [55]:
print(newton_1d(1, f, df, ddf))
print(newton_1d(2, f, df, ddf))
print(newton_1d(3, f, df, ddf))
print(newton_1d(4, f, df, ddf))
print(newton_1d(5, f, df, ddf))
print(newton_1d(6, f, df, ddf))

Iterations = 3
(1.0, 1.5707963267954879)
Iterations = 2
(0.9999999999704976, 1.5708040082580965)
Iterations = 4
(1.0, -4.712388980650428)
Iterations = 3
(-1.0, 4.712388980912051)
Iterations = 2
(-0.9999999999999831, 4.712389164306483)
Iterations = 4
(0.9999999975153049, 1.5707258328931568)


## Двумерный демпфированный метод Ньютона

$$
f(x) = x^\intercal Ax + b^\intercal x + x_1^4 + x_2^4 \to \min
$$
$$
\nabla f(x) = (A^\intercal + A)x + b + 4\begin{pmatrix}x_1^3\\x_2^3\end{pmatrix}
$$
$$
\nabla^2 f(x) = (A^\intercal + A) + 12\begin{pmatrix}x_1^2 & 0 \\ 0 & x_2^2\end{pmatrix}
$$

In [184]:
def generate_f():
    A = np.random.random((2, 2))
    b = np.random.random((2,))
    def f(x):
        return np.dot(np.dot(x, A), x) + np.dot(b, x) + x[0]**4 + x[1]**4
    def df(x):
        return np.dot((A + np.transpose(A)), x) + b + 4*x**3
    def ddf(x):
        return np.transpose(A) + A + 12 * np.array([[x[0]**2, 0], [0, x[1]**2]])
    return f, df, ddf, A, b

In [181]:
f, df, ddf, A, b = generate_f()

In [182]:
ALPHA0 = 1
THETA = 0.5
EPSILON = 0.1
GAMMA = 0.01          #regularization

def get_step(x, f, df, ddf, iter_max=100):
    alpha = ALPHA0
    iters = 0
    s = -df(x)
    while (iters == 0 or lhs > rhs) and iters < iter_max:
        lhs = f(x + alpha*s)
        rhs = f(x) - EPSILON*alpha*np.dot(s, s)
        if lhs > rhs:
            alpha = alpha*THETA
        iters +=1
    return alpha

def is_invertible(A):
    return A.shape[0] == A.shape[1] and np.linalg.matrix_rank(A) == A.shape[0]

def regularize(A):
    return A + np.eye(A.shape)*GAMMA
    
def newton_demph(x_0, f, df, ddf, eps=0.0001, iter_max=1000000, verbose=True):
    """
    Input:
    x_0         starting point
    f           target function
    df          gradient function
    ddf         hessian function
    eps         stopping treshold for gradient norm
    iter_max    maximum number of iterations
    verbose     enabling text output
    
    Output:
    function extremum value, extremum point
    """
    x =x_0
    x_old = None
    iters = 0
    while iters < iter_max and np.linalg.norm(df(x)) > eps:
        step = get_step(x, f, df, ddf)
        x_old = x
        hess = ddf(x)
        if not is_invertible(hess):
            hess = regularize(hess)
        x = x_old - step * np.dot(np.linalg.inv(hess), df(x))
        iters += 1
    if verbose:
        print("Iterations: %d" % iters)
    return f(x), x

In [185]:
print(newton_demph(np.array([0, 0]), f, df, ddf))
print(newton_demph(np.array([0, 1]), f, df, ddf))
print(newton_demph(np.array([1, 0]), f, df, ddf))
print(newton_demph(np.array([1, 1]), f, df, ddf))

Iterations: 25
(-0.18185595649329717, array([ 0.0738453 , -0.38911844]))
Iterations: 19
(-0.18185595654439501, array([ 0.07383649, -0.38911777]))
Iterations: 12
(-0.18185595754158954, array([ 0.07384557, -0.38914327]))
Iterations: 20
(-0.18185595657547787, array([ 0.07387361, -0.38913549]))
